In [2]:
import glob
import pandas as pd
import altair as alt

# load in csv files from abstract simulations
abstract_results = glob.glob('results/abstract/*')
results = sorted(abstract_results)

# create a list to store each the dataframe loaded from each csv
results_lst = []
for filename in results:
    df = pd.read_csv(filename, index_col=0, header=0)
    max_epochs = filename.split('_')[-1][:-4]  # extract max epochs
    df['Max Epochs'] = max_epochs
    results_lst.append(df)

# combine csv files using pandas
abstract_results_df = pd.concat(results_lst, axis=0, ignore_index=True)

# create the base plotting object
# mark_point() specifies to plot each data point
# mark_line() specifies to plot lines between points
# encode() and its arguments specify which columns from the data frame to read from and plot
base = alt.Chart(abstract_results_df).mark_point(filled=True).mark_line().encode(
    x=alt.X('Epoch'),
    y=alt.Y('Probability Correct', scale=alt.Scale(domain=(0, 1))),
    color='Type:N'
)

# create filters for each variable
models = ['resnet18', 'resnet152']
model_dropdown = alt.binding_radio(options=models)
model_select = alt.selection_single(fields=['Model'], bind=model_dropdown, name='model')

types = [1, 2, 3, 4, 5, 6]
type_dropdown = alt.binding_radio(options=types)
type_select = alt.selection_single(fields=['Type'], bind=type_dropdown, name="type")

losses = ['ll', 'hinge']
loss_dropdown = alt.binding_radio(options=losses)
loss_select = alt.selection_single(fields=['Loss Type'], bind=loss_dropdown, name='loss')

epochs = ['16', '32', '64', '128']
epochs_dropdown = alt.binding_radio(options=epochs)
epochs_select = alt.selection_single(fields=['Max Epochs'], bind=epochs_dropdown, name='epochs')

# add any additional _selects necessary inside add_selection
# and then add a separate transform_filter() for each select object
interactive_plot = base.add_selection(
    loss_select, epochs_select
).transform_filter(
    loss_select
).transform_filter(
    epochs_select    
).properties(title='Abstract Stimuli')

# display the plot in the notebook
# you may need to select all of the options once the plot is created to see something sensible
# it would be good to find a way to set a default value to plot initially
interactive_plot

alt.Chart(...)